<a href="https://colab.research.google.com/github/ajaypr55/kaggle/blob/master/avazu_ctr_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ajay88","key":"e59115f39668b085910a90197e69599f"}'}

In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 62 Dec 28 12:08 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!mkdir data

adc.json  data	kaggle.json  log  model  sample_data


In [0]:
""# List available datasets.
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
spscientist/students-performance-in-exams                     Students Performance in Exams                         8KB  2018-11-09 18:25:25           8388  
mehdidag/black-friday                                         Black Friday                                          5MB  2018-07-25 20:49:48          23281  
szamil/who-suicide-statistics                                 WHO Suicide Statistics                              307KB  2018-08-29 21:23:11           6031  
lava18/google-play-store-apps                                 Google Play Store Apps                                2MB  2018-09-18 20:49:49          28577  
iarunava/happy-house-dataset                        

In [0]:
!ls

adc.json  data	kaggle.json  log  model  sample_data


In [0]:
# Copy the stackoverflow data set locally.
!kaggle competitions download -p data/ avazu-ctr-prediction

  0% 0.00/498k [00:00<?, ?B/s]
100% 498k/498k [00:00<00:00, 34.5MB/s]
  0% 0.00/594k [00:00<?, ?B/s]
100% 594k/594k [00:00<00:00, 39.0MB/s]
 99% 1.33G/1.35G [00:11<00:00, 134MB/s]
100% 1.35G/1.35G [00:12<00:00, 120MB/s]
100% 4.14G/4.16G [00:45<00:00, 55.1MB/s]
100% 4.16G/4.16G [00:45<00:00, 97.2MB/s]


In [0]:
!cd data && unzip -o -q train.zip && unzip -o -q test.zip && cd ..

In [0]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [0]:
train_df = pd.read_csv("data/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [0]:
def prepareImages(data, m):
    print("Preparing images")
    X_train = np.zeros((m, 100, 100, 3))
    count = 0
    
    for fig in data['Image']:
        #load images into images of size 100x100x3
        img = image.load_img("data/"+"/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)

        X_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [0]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    # print(y.shape)
    return y, label_encoder

In [0]:
X = prepareImages(train_df, train_df.shape[0])
X /= 255

Preparing images
Processing image:  1 ,  0000e88ab.jpg
Processing image:  501 ,  04c72257b.jpg
Processing image:  1001 ,  09cacb84d.jpg
Processing image:  1501 ,  0ef961892.jpg
Processing image:  2001 ,  141b56a1a.jpg
Processing image:  2501 ,  199a417aa.jpg
Processing image:  3001 ,  1ec170983.jpg
Processing image:  3501 ,  23f084b93.jpg
Processing image:  4001 ,  29163ad0b.jpg
Processing image:  4501 ,  2e0fab120.jpg
Processing image:  5001 ,  3347515d9.jpg
Processing image:  5501 ,  3842d71dc.jpg
Processing image:  6001 ,  3d7f4c7d5.jpg
Processing image:  6501 ,  425f763ca.jpg
Processing image:  7001 ,  4714400cd.jpg
Processing image:  7501 ,  4c082fbdf.jpg
Processing image:  8001 ,  50c683e23.jpg
Processing image:  8501 ,  560d986ad.jpg
Processing image:  9001 ,  5b68c83ed.jpg
Processing image:  9501 ,  60410f111.jpg
Processing image:  10001 ,  654951f81.jpg
Processing image:  10501 ,  6a572256c.jpg
Processing image:  11001 ,  6f96f55b6.jpg
Processing image:  11501 ,  74da2b511.jpg

In [0]:
y, label_encoder = prepare_labels(train_df['Id'])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
y.shape

(25361, 5005)

In [0]:
model = Sequential()

model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

model.add(BatchNormalization(axis = 3, name = 'bn0'))
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2), name='max_pool'))
model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3), name='avg_pool'))

model.add(Flatten())
model.add(Dense(500, activation="relu", name='rl'))
model.add(Dropout(0.8))
model.add(Dense(y.shape[1], activation='softmax', name='sm'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 94, 94, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 94, 94, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 94, 94, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 47, 47, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 45, 45, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 45, 45, 64)        0         
_________________________________________________________________
avg_pool (AveragePooling2D)  (None, 15, 15, 64)        0         
__________

In [0]:
history = model.fit(X, y, epochs=100, batch_size=100, verbose=1)
gc.collect()

Epoch 1/100
25361/25361 [==============================] - 31s 1ms/step - loss: 6.1869 - acc: 0.3796
Epoch 2/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.9180 - acc: 0.3811
Epoch 3/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.8108 - acc: 0.3811
Epoch 4/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.7412 - acc: 0.3811
Epoch 5/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.7129 - acc: 0.3811
Epoch 6/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.6802 - acc: 0.3811
Epoch 7/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.6474 - acc: 0.3811
Epoch 8/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.6091 - acc: 0.3811
Epoch 9/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.5802 - acc: 0.3811
Epoch 10/100
25361/25361 [==============================] - 27s 1ms/step - loss: 5.5456 - a

NameError: ignored

In [0]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

NameError: ignored

In [0]:
!mkdir data/test

In [0]:
!cp data/test.zip data/test/

In [0]:
!cd data/test && unzip -o -q test.zip && cd ..

MessageError: ignored

In [0]:
!ls data/test/

00028a005.jpg  330aa20b1.jpg  65cb74051.jpg  99780e707.jpg  cc365a6ac.jpg
000dcf7d8.jpg  330fbb0bc.jpg  65cb8a649.jpg  9983ab219.jpg  cc37808a9.jpg
000e7c7df.jpg  331399819.jpg  65cba7e5a.jpg  99968e3be.jpg  cc3b01499.jpg
0019c34f4.jpg  331c42937.jpg  65ce604cc.jpg  999f99082.jpg  cc47b0982.jpg
001a4d292.jpg  331c9709d.jpg  65e9fd59e.jpg  99a6756a0.jpg  cc47d1aa0.jpg
00247bc36.jpg  332a57fe7.jpg  65eb43ef7.jpg  99a765134.jpg  cc5360c6f.jpg
0027089a4.jpg  332ab8314.jpg  65ee8c179.jpg  99afdd9f8.jpg  cc54527f2.jpg
002de4d94.jpg  332e56664.jpg  65ee9bbf6.jpg  99b1e4de0.jpg  cc56f2c79.jpg
002f52f0c.jpg  3337e3aa3.jpg  65f637a56.jpg  99b4ec13e.jpg  cc6e079a8.jpg
002fd89d4.jpg  33380ae50.jpg  65fd236f0.jpg  99e6e5232.jpg  cc70e7bd7.jpg
00313e2d2.jpg  3344130ad.jpg  65fff5e87.jpg  99fea0b90.jpg  cc739b946.jpg
00379666f.jpg  3351f4f65.jpg  660049e53.jpg  9a00976a0.jpg  cc75d8c29.jpg
0041a9867.jpg  3356cf4fc.jpg  660352b03.jpg  9a0e710b3.jpg  cc762858e.jpg
004344e9f.jpg  3369e288f.jpg  66043157

In [0]:
test = os.listdir("data/test/")
print(len(test))

NameError: ignored

In [0]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

NameError: ignored

In [0]:
def prepareImagesTest(data, m):
    print("Preparing images")
    X_train = np.zeros((m, 100, 100, 3))
    count = 0
    
    for fig in data['Image']:
        #load images into images of size 100x100x3
        img = image.load_img("data/test"+"/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)

        X_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [0]:
!rm data/test/test.zip

In [0]:
X = prepareImagesTest(test_df, test_df.shape[0])
X /= 255

NameError: ignored

In [0]:
predictions = model.predict(np.array(X), verbose=1)


In [0]:
for i, pred in enumerate(predictions):
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))


In [0]:
test_df.head(10)
test_df.to_csv('submission.csv', index=False)